In [1]:
import pandas as pd
from typing import Dict, List
from collections import Counter
from pprint import pprint
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
sns.set()

import pandas as pd
from itertools import combinations
from pprint import pprint
from typing import Iterable, List
from collections import defaultdict


In [2]:
path1="/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/data/Dataset1.csv"
path2="/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/data/Dataset2.csv"
path3="/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/data/Dataset3.xlsx"

In [3]:
df=pd.read_excel(path3)

In [4]:
df

,Temperature,Humidity,Rainfall,Soil,Crop,Fertilizer
0,24.87,82.84,295.61,Clayey,rice,DAP
1,28.69,96.65,178.96,laterite,Coconut,Good NPK
2,20.27,81.64,270.44,silty clay,rice,MOP
3,25.07,95.02,192.90,sandy,Coconut,Urea
4,25.04,95.90,174.80,coastal,Coconut,Urea
...,...,...,...,...,...,...
290,26.07,96.70,143.53,sandy,Coconut,MOP
291,25.63,83.53,209.90,silty clay,rice,MOP
292,24.52,83.54,230.45,Clayey,rice,MOP
293,25.66,83.47,217.38,Clayey,rice,MOP


In [5]:
df['Soil'].unique()

array(['Clayey', 'laterite', 'silty clay', 'sandy', 'coastal',
       'clay loam', 'alluvial'], dtype=object)

In [6]:
df['Crop'].unique()

array(['rice', 'Coconut'], dtype=object)

In [7]:
df['Crop'].unique()

array(['rice', 'Coconut'], dtype=object)

In [8]:
df['Fertilizer'].unique()

array(['DAP', 'Good NPK', 'MOP', 'Urea'], dtype=object)

# Descritisize the data

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Temperature  295 non-null    float64
 1   Humidity     295 non-null    float64
 2   Rainfall     295 non-null    float64
 3   Soil         295 non-null    object 
 4   Crop         295 non-null    object 
 5   Fertilizer   295 non-null    object 
dtypes: float64(3), object(3)
memory usage: 14.0+ KB


In [10]:
def calculate_central_tendencies(df : pd.DataFrame) -> Dict[str,Dict[str,float]]:
    """this function calculate the central tendencies : mean , median , mode , min , max of each column in the dataframe
    Args:
        df (pd.Dataframe): pandas dataframe to calculate the central tendencies

    Returns:
        Dict: dictionary of the central tendencies of each column
    """
    tendencies={}
    new_df=df.copy()

    for column in new_df.columns:
        flag=False
        if new_df[column].dtype=='object' or new_df[column].dtype=='string':
            tendencies[column] = {'min': None, 'max': None, 'mean': None, 'median': None, 'mode': None, 'std': None}
        else:
            if new_df[column].dtype=='datetime64[ns]':
                flag=True
                new_df[column]=new_df[column].astype(np.int64)
                
            #calculate mean 
            col_len=len(new_df[column])
            mean=sum(new_df[column])/col_len

            #calculate median
            sorted_column=sorted(new_df[column])
            if col_len%2==0:
                median1 = sorted_column[col_len//2]
                median2 = sorted_column[col_len//2-1]
                median=(median1+median2)/2
            else:
                median=sorted_column[col_len//2]

            #calculate mode
            counter = Counter(new_df[column])
            mode = counter.most_common(1)[0][0] 

            #calculate min    
            minimum=sorted_column[0]

            #calculate max
            maximum=sorted_column[-1]

            #calculate standard deviation
            std=0
            for data in new_df[column]:
                std+=((data-mean)**2)
            std=std/col_len
            std=std**0.5

            if df[column].dtype=='datetime64[ns]':
                mean, median, mode, minimum, maximum, std = pd.to_datetime([mean, median, mode, minimum, maximum, std])
                flag=False

            tendencies[column]={'min':minimum,'max':maximum,'mean':mean,'median':median,'mode':mode,'std':std}
            
    return tendencies

In [11]:
df.iloc[294]

Temperature         23.83
Humidity            84.81
Rainfall           298.56
Soil           silty clay
Crop                 rice
Fertilizer            MOP
Name: 294, dtype: object

In [409]:
r=calculate_central_tendencies(df)

In [410]:
def assigning_classes_equal_freq(df : pd.DataFrame,column : str, k : int) -> List[int]:
    """ this function assign classes to the dataframe column based on the number of classes k

    Args:
        df (pd.DataFrame): dataframe to assign classes to
        column (str): column to assign classes to
        k (int): number of classes

    Returns:
        List[int]: list of classes assigned to each row in the dataframe
    """
    
    new_df=df.copy()
    sorted_data = new_df[column].sort_values()
    
    observations_per_class = len(sorted_data) // k
    
    class_assignments = []
    current_class = 1
    current_observations = 0
    
    for i, _ in enumerate(sorted_data):
        class_assignments.append(current_class)
        current_observations += 1
    
        if i == len(sorted_data) - 1:
            break
        
        if current_observations == observations_per_class and current_class < k:
            current_class += 1
            current_observations = 0
    
    if len(sorted_data) % k != 0:
        remaining_observations = len(sorted_data) - observations_per_class * (k - 1)
        class_assignments.extend([k] * remaining_observations)
    
    return class_assignments

In [411]:
r=assigning_classes_equal_freq(df,'Temperature',5)

In [6]:
def assigning_classes_equal_width(df : pd.DataFrame, column : str, k : int) -> List[int]:
    """ this function assign classes to the dataframe column based on the number of classes k with equal width

    Args:
        df (pd.DataFrame): dataframe to assign classes to
        column (str): column to assign classes to
        k (int): number of classes

    Returns:
        List[int]: list of classes assigned to each row in the dataframe
    """

    sorted_data = df['Temperature'].sort_values()

    range_temperature = sorted_data.max() - sorted_data.min()

    class_width = range_temperature / k

    class_assignments = []
    current_class = 1
    current_boundary = sorted_data.min() + class_width

    for observation in sorted_data:
        if observation > current_boundary:
            current_class += 1
            current_boundary += class_width

            if current_boundary > sorted_data.max():
                current_boundary = sorted_data.max()
                current_class = current_class-1

        class_assignments.append(current_class)

    return class_assignments

In [16]:
a=assigning_classes_equal_width(df,'Temperature',7)

In [17]:
df['Temperature']=assigning_classes_equal_width(df,'Temperature',7)


In [18]:
df['Temperature'].value_counts()

Temperature
6    85
4    69
7    58
2    44
1    37
3     1
5     1
Name: count, dtype: int64

In [415]:
class AssociationRuleMiner:
    def __init__(self,min_support: int = 4, threshold: float = 1):
        self.min_support = min_support
        if threshold > 1 or threshold < 0:
            raise ValueError("The threshold must be between 0 and 1")
        self.threshold = threshold

    def execute(self, data: str, transactions_groups: List[str] , items_groups: List[str]):

        self.df = data.copy()
        self.transaction_columns = transactions_groups
        self.items_groups = items_groups

        transactions = self.rearrange_data()

        frequent_itemsets, k = self.apriori(transactions["Items"])

        generated_rules = self.generate_association_rules(frequent_itemsets[k])
        self.find_confidence(generated_rules, transactions["Items"])

    def rearrange_data(self) -> pd.DataFrame:
        """ this function is responsible for rearranging the data to be in the form of a transactional dataframe

        Returns:
            pd.DataFrame: a transactional dataframe 
        """
        # Melt the DataFrame to convert it from wide to long format
        melted_df = pd.melt(self.df, id_vars=self.transaction_columns, value_vars=self.items_groups)

        # Drop rows with NaN values (assuming NaNs are not valid items in your context)
        melted_df = melted_df.dropna()


        # Group by the transaction column and aggregate items into a set
        result_df = (
            melted_df.groupby(self.transaction_columns[0])
            .agg({"value": lambda x: set(x)})
            .sort_values(by="value", key=lambda x: x.apply(len), ascending=False)
            .rename(
                columns={
                    self.transaction_columns[0]: "Transaction",
                    "value": "Items",
                }
            )
        )

        return result_df

    def generate_items(self, transactions: pd.Series) -> set:
        """ this function is responsible for generating the unique items from the transactions

        Args:
            transactions (pd.Series): a list of transactions

        Returns:
            set: a set containing the unique items
        """
        # generate the unique items from the transactions
        items = set()
        for transaction in transactions:
            items.update(transaction)
        return items


    def generate_candidates(self, items, k) -> list[set]:
        """ this function is responsible for generating the candidates from the items

        Args:
            items (_type_): a list of items
            k (_type_): a number representing the cardinality of the candidates

        Returns:
            list[set]: a list of candidates
        """
        # generate the candidates from the items
        # the candidates are the combinations of the items
        result = []
        for combination in combinations(items, k):
            result.append(set(combination))
        return result
    
    def support(self, transactions: Iterable[set], candidate: set) -> int :
        """ this function is responsible for calculating the support of a candidate

        Args:
            transactions (Iterable[set]): a list of transactions
            candidate (set): a candidate

        Returns:
            int : the support of the candidate 
        """
        # calculate the support of a candidate
        count = 0
        for transaction in transactions:
            if candidate.issubset(transaction):
                count += 1
        return count
    
    def select_adequat_candidats(self, transactions: Iterable[set], candidates: list[set]) -> dict[dict, int]:
        """ this function is responsible for selecting the candidates that have a support greater than the minimum support

        Args:
            transactions (Iterable[set]): a list of transactions
            candidates (list[set]): a list of candidates

        Returns:
            dict[dict, int]: a dictionary containing the candidates and their support
        """
        # select the candidates that have a support greater than the minimum support
        candidat_support = defaultdict(int)
        for transaction in transactions:
            for candidat in candidates :
                # if the candidate is a subset of the transaction
                if set(candidat).issubset(transaction):
                    # increment the support of the candidate
                    candidat_support[tuple(candidat)] += 1
        return {
            candidat: support
            for candidat, support in candidat_support.items()
            if support >= self.min_support
        }

    def apriori(self, transactions: Iterable[set]) -> tuple[dict[int, set], int]:
        """ this function is responsible for generating the frequent itemsets from the transactions

        Args:
            transactions (Iterable[set]): a list of transactions

        Returns:
            tuple[dict[int, set], int]: a tuple containing the frequent itemsets and the maximum k
        """
        items = self.generate_items(transactions)
        candidates = self.generate_candidates(items, 1)
        candidats_support = self.select_adequat_candidats(transactions, candidates)
        if not candidats_support:
            # if there is no candidate with a support greater than the minimum support return an empty set
            return {1: set()}, 1

        # generate the items from the support counts
        items = self.generate_items(candidats_support)

        k = 2
        frequent_itemsets = {}

        while True:
            # generate the candidates
            candidates = self.generate_candidates(items, k)
            # if there is no candidate break the loop
            if not candidates:
                break
            print("C",k,"=",candidates)

            # select the candidates that have a support greater than the minimum support
            candidats_support = self.select_adequat_candidats(transactions, candidates)
            # if there is no candidate break the loop
            if not candidats_support:
                break        
            print("L",k,"=",candidats_support)
            # generate the items from the support counts
            items = self.generate_items(candidats_support)
            frequent_itemsets[k] = items
            k += 1

        return frequent_itemsets, k - 1

    def generate_association_rules(self, frequent_itemsets: set) -> defaultdict[frozenset, set]:
        """ this function is responsible for generating the association rules from the frequent itemsets

        Args:
            frequent_itemsets (set): a set containing the frequent itemsets

        Returns:
            defaultdict[frozenset, set]: a dictionary containing the association rules
        """
        max_k = len(frequent_itemsets)
        rules = defaultdict(set)

        for k in range(1, max_k):
            transactions = list(combinations(frequent_itemsets, k))
            for j in range(1, max_k):
                candidates_transaction = list(combinations(frequent_itemsets, j))
                for translation in transactions:
                    for candidate in candidates_transaction:
                        if not set(candidate).issubset(translation):
                            rules[frozenset(translation)].add(candidate)
        return rules

    def find_confidence(self, rules: dict[frozenset, set], transactions: Iterable[set]) -> None:
        """ this function is responsible for finding the confidence of the association rules

        Args:
            rules (dict[frozenset, set]): a dictionary containing the association rules
            transactions (Iterable[set]): a list of transactions
            threshold (float, optional): the threshold if the confidence  . Defaults to 0.5.
        """
        # for each transaction in the rules
        for transaction, candidates in rules.items():
            # for each candidate in the transaction
            for candidate in candidates:
                # calculate the confidence of the candidate
                # the confidence is the support of the union of the candidate and the transaction divided by the support of the transaction
                try : 
                    union = transaction.union(candidate)
                    confidence = self.support(transactions, union) / self.support(
                        transactions, transaction
                    )
                    # if the confidence is greater than the threshold print the association rule
                    if confidence >= self.threshold:
                        print(f"{list(transaction)} => {candidate} : {confidence:.2f}")
                except Exception as e : 
                    pass



In [417]:
df

,Temperature,Humidity,Rainfall,Soil,Crop,Fertilizer
0,24.87,82.84,295.61,Clayey,rice,DAP
1,28.69,96.65,178.96,laterite,Coconut,Good NPK
2,20.27,81.64,270.44,silty clay,rice,MOP
3,25.07,95.02,192.90,sandy,Coconut,Urea
4,25.04,95.90,174.80,coastal,Coconut,Urea
...,...,...,...,...,...,...
290,26.07,96.70,143.53,sandy,Coconut,MOP
291,25.63,83.53,209.90,silty clay,rice,MOP
292,24.52,83.54,230.45,Clayey,rice,MOP
293,25.66,83.47,217.38,Clayey,rice,MOP


In [416]:
miner = AssociationRuleMiner(min_support=4,threshold=1)
miner.execute(df,transactions_groups=['Soil'],items_groups=['Crop','Fertilizer'])

C 2 = [{'Good NPK', 'DAP'}, {'Good NPK', 'Urea'}, {'Good NPK', 'Coconut'}, {'Good NPK', 'MOP'}, {'DAP', 'Urea'}, {'DAP', 'Coconut'}, {'DAP', 'MOP'}, {'Urea', 'Coconut'}, {'Urea', 'MOP'}, {'MOP', 'Coconut'}]
L 2 = {('Good NPK', 'DAP'): 6, ('Good NPK', 'Urea'): 6, ('Good NPK', 'MOP'): 6, ('DAP', 'Urea'): 7, ('DAP', 'MOP'): 7, ('Urea', 'MOP'): 7, ('DAP', 'Coconut'): 4, ('Urea', 'Coconut'): 4, ('MOP', 'Coconut'): 4}
C 3 = [{'Good NPK', 'DAP', 'Urea'}, {'Good NPK', 'DAP', 'Coconut'}, {'Good NPK', 'DAP', 'MOP'}, {'Good NPK', 'Urea', 'Coconut'}, {'Good NPK', 'Urea', 'MOP'}, {'Good NPK', 'MOP', 'Coconut'}, {'DAP', 'Urea', 'Coconut'}, {'DAP', 'Urea', 'MOP'}, {'DAP', 'MOP', 'Coconut'}, {'Urea', 'Coconut', 'MOP'}]
L 3 = {('Good NPK', 'DAP', 'Urea'): 6, ('Good NPK', 'DAP', 'MOP'): 6, ('Good NPK', 'Urea', 'MOP'): 6, ('DAP', 'Urea', 'MOP'): 7, ('DAP', 'Urea', 'Coconut'): 4, ('DAP', 'MOP', 'Coconut'): 4, ('Urea', 'Coconut', 'MOP'): 4}
C 4 = [{'Good NPK', 'DAP', 'Urea', 'Coconut'}, {'Good NPK', 'DAP',